In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

json = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

postsDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
postsDf = pd.DataFrame(columns = postscolumns)

In [3]:
import time

time_now = int(time.time())
for event in json:
    if int(datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print(str(err))
            location = None

        try:
        
            new_row = pd.DataFrame({
                    "account": ["WUSA"],
                    "date": [time_now],
                    "caption": str(event["description"])+" [For More Information, Click View Post] ",
                    "display_photo": event["image"],
                    "url": [event['url']],
                    "likes": [0],
                    "event_details": [{
                        "is_event": True,
                        "event_name": event["name"],
                        "event_description": str(event["description"])+" ... ",
                        "categories": [["SOCIAL"]],
                        "start_time": int(datetime.fromisoformat(event["startDate"]).timestamp()+ 7200),
                        "end_time": int(datetime.fromisoformat(event["endDate"]).timestamp()+ 7200),
                        "location": location,
                    }]
            })

            postsDf = pd.concat([postsDf, new_row])
        except:
            print("failure")

print(postsDf)
postsDf.to_csv(path_or_buf="WUSA events")

failure
'streetAddress'
  account        date                                            caption  \
0    WUSA  1729044839  🥳 Raise your steins and celebrate Oktoberfest ...   
0    WUSA  1729044839  Cozy up with some tea 🍵and meet fellow Waterlo...   
0    WUSA  1729044839  👻 Come join us for a fun evening at our Pumpki...   
0    WUSA  1729044839  Need a pick me up for the Monday blues? Join u...   
0    WUSA  1729044839  Still on the hunt for the perfect Halloween co...   
0    WUSA  1729044839  Get your game on! Join us for our weekly board...   
0    WUSA  1729044839  Learn how to service your bike at our weekly w...   
0    WUSA  1729044839  📢 Come join Mates for a cozy, snack-filled, st...   
0    WUSA  1729044839  Looking for a fun way to connect with fellow s...   
0    WUSA  1729044839  Join us every Monday in the Glow Centre for ou...   
0    WUSA  1729044870  🥳 Raise your steins and celebrate Oktoberfest ...   
0    WUSA  1729044870  Cozy up with some tea 🍵and meet fellow Wa

In [4]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with these IDs
result = collection.delete_many({'account': "WUSA"})

# Print the number of documents deleted
print(f"Documents removed: {result.deleted_count}")


Documents removed: 0


In [5]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

for index, row in postsDf.iterrows():
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    row_dict = {}
    for column in postsDf.columns.tolist():
        row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)  
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")
    insertObjectIds.append(result.inserted_id)
    time.sleep(0.5)


Inserted document ID: 670f2194a39953f6293845cd
Inserted document ID: 670f2197a39953f6293845ce
Inserted document ID: 670f2197a39953f6293845cf
Inserted document ID: 670f2198a39953f6293845d0
Inserted document ID: 670f219aa39953f6293845d1
Inserted document ID: 670f219aa39953f6293845d2
Inserted document ID: 670f219ba39953f6293845d3
Inserted document ID: 670f219ca39953f6293845d4
Inserted document ID: 670f219da39953f6293845d5
Inserted document ID: 670f219ea39953f6293845d6
Inserted document ID: 670f219fa39953f6293845d7
Inserted document ID: 670f21a0a39953f6293845d8
Inserted document ID: 670f21a1a39953f6293845d9
Inserted document ID: 670f21a2a39953f6293845da
Inserted document ID: 670f21a3a39953f6293845db
Inserted document ID: 670f21a4a39953f6293845dc
Inserted document ID: 670f21a5a39953f6293845dd
Inserted document ID: 670f21a6a39953f6293845de
Inserted document ID: 670f21a7a39953f6293845df
Inserted document ID: 670f21a8a39953f6293845e0
Inserted document ID: 670f21a9a39953f6293845e1
